# Out-of-sample analysis - am example
### Mixture of CVaR's with Sharpe type of optimization

This is an illustration of *out-of-sample analysis* workflow. It is also known as *back testing* or *historical simulation*.

The example presented here is for a mCVaR (a mixture of 3 CVaR's) with Sharpe type of optimization (*i.e.* the portfolio that maximizes the mCVaR-Sharpe ratio).

Similar results can be obtained for any other implemented risk measure and type of optimization.

We start by importing the **azapy** and other useful packages.

In [16]:
import numpy as np
import pandas as pd

import sys
sys.path.append("..")
import azapy as az

### Collect historical market data for a set of portfolio components.

- `symb` is the list of stock symbols (portfolio components).
- `sdate` and `edate` are the start and end dates of historical time-series.
- `mktdir` is the name of the directory used to save the market data collected from *alphavantage*.
    
> Note: if the flag `force=False` then a reading from `dir=mktdir` is attempted. If it fails, then the *alphavantage* servers will be accessed. The new data will be saved to the `dir=mktdir` (*for more information see the readMkT documentation*).

In [17]:
symb = ['GLD', 'TLT', 'XLV', 'VGT', 'PSJ']

sdate = pd.to_datetime("2012-01-01")
edate = pd.to_datetime("today")

mktdir = "../MkTdata"

mktdata = az.readMkT(symb, dstart = sdate, dend = edate, dir=mktdir, force=False) 

Read GLD form ../MkTdata
Read TLT form ../MkTdata
Read XLV form ../MkTdata
Read VGT form ../MkTdata
Read PSJ form ../MkTdata


### Set the mixture of CVaR's parameters

- `alpha` is the list of CVaR confidence levels,
- `coef` is the list of CVaR's weights (coefficients). 

Note: `len(coef)=len(alpha)`. All the coefficients must be `>0`.

In [18]:
alpha = [0.95, 0.90, 0.85]
coef = [0.2, 0.3, 0.5]

### Set the Port_CVaR class.

For a full list of available setting parameters see the documentation.

In [19]:
pr1 = az.Port_CVaR(mktdata, pname='CVaRPort')

### Set the model parameters 

> Note: This method must be called before any other. It will perform the main historical simulation.

For a full list of available setting parameters see the documentation.

In [20]:
port = pr1.set_model(mu=0., alpha=alpha, coef=coef)

### View the portfolio time-series 

For reference EMA for 30 and 200 business days were added *(default behavior)*.

> Note the use of flag `fancy=True`. It designates the use of *plotly* package for interactive time-series. 

For a full list of available setting parameters see the documentation.

In [21]:
_ = pr1.port_view(fancy=True)

### View the portfolio and its components time-series (in a relative bases)

For a full list of available setting parameters see the documentation.

In [22]:
_ = pr1.port_view_all(fancy=True)

### Portfolio weights for each rebalancing period 

- `Droll` is the rolling date. The new positions are acquired at the end of trading day (theoretically at the closing price).
- `Dfix` is the fixing date relative to which the weights are computed. This is also the last closing prices included in the evaluation of the weights.
- `Dhist` this is the earliest date included in the calibration of weights. The hist data used in the calibration is between and including the `Dhist` and `Dfix`.

> Note the use of flag `fancy=True` - the weights are reported in percent with 2 decimals. 

For a full list of available setting parameters see the documentation.

In [23]:
pr1.get_weights(fancy=True)

,Droll,Dfix,Dhist,GLD,PSJ,TLT,VGT,XLV
0,2015-06-25,2015-06-24,2012-03-23,0.00,0.00,10.74,0.00,89.26
1,2015-09-25,2015-09-24,2012-06-22,0.00,0.00,13.12,0.00,86.88
2,2015-12-28,2015-12-24,2012-09-24,0.00,35.48,34.84,11.61,18.07
3,2016-03-28,2016-03-24,2012-12-24,0.00,0.39,40.35,21.04,38.22
4,2016-06-27,2016-06-24,2013-03-22,0.00,16.09,42.00,12.00,29.92
5,2016-09-27,2016-09-26,2013-06-26,0.00,22.59,48.18,4.66,24.56
6,2016-12-27,2016-12-23,2013-09-23,0.00,0.00,38.86,61.14,0.00
7,2017-03-28,2017-03-27,2013-12-27,0.00,0.00,36.89,63.11,0.00
8,2017-06-27,2017-06-26,2014-03-26,0.00,0.00,36.02,63.98,0.00
9,2017-09-26,2017-09-25,2014-06-25,0.00,6.08,35.82,58.11,0.00


### Portfolio performance

Reports the portfolio and its components performance for the duration of the investment.

- `RR` is the portfolio average annual rate of returns.
- `DD` is the maximum drawdown rate.
- `Beta` is the ratio `RR/DD`.
- `DD_date` is the date of maximum drawdown.
- `DD_start` is the date when the maximum drawdown event had started.
- `DD_end` is the date when the maximum drawdown event had ended. If it is `Nan`, then the drawdown is still in progress (`DD` and `DD_date` are only provisional).

> Note the use of flag `fancy=True` - the rates are reported in percent with 2 decimals. 

In [24]:
pr1.port_perf(fancy=True)

,RR,DD,Beta,DD_date,DD_start,DD_end
symbol,,,,,,
CVaRPort,9.75,-18.30,0.532649,2020-03-20,2020-02-19,2020-06-23
VGT,23.24,-31.86,0.729357,2020-03-23,2020-02-19,2020-06-09
PSJ,22.12,-30.69,0.720979,2020-03-16,2020-02-19,2020-05-22
XLV,16.74,-28.39,0.589717,2020-03-23,2020-01-22,2020-07-15
TLT,4.92,-21.34,0.230336,2021-03-18,2020-08-04,NaN
GLD,0.81,-42.11,0.019280,2015-12-17,2012-10-04,2020-07-22


### Portfolio drawdowns 

Reports the largest drawdown events. The default is the first 5. 

> Note the use of flag fancy=True - the rates are reported in percent with 2 decimals.

For a full list of available setting parameters see the documentation.

In [25]:
pr1.port_drawdown(fancy=True)

,DD,Date,Start,End
No,,,,
1,-18.30,2020-03-20,2020-02-19,2020-06-23
2,-15.73,2018-12-24,2018-08-31,2019-02-25
3,-14.62,2015-09-28,2015-08-05,2016-06-02
4,-10.58,2021-03-08,2020-08-06,NaN
5,-9.91,2019-10-22,2019-07-26,2020-01-03


### Portfolio annual returns 

Annual returns are per calendar year (1 Jan to 31 Dec). If the first or last years are incomplete, then the rates are reported per invested period in the respective years. 

> Note the use of flag fancy=True - the rates are reported in percent with 2 decimals.

In [26]:
pr1.port_annual_returns(fancy=True)

,CVaRPort
year,
2015,-2.17%
2016,1.30%
2017,26.06%
2018,0.28%
2019,24.15%
2020,11.88%
2021,-0.15%


### Portfolio monthly returns

Same as above but per calendar month of each year. This are simple monthly rates. 

> Note the use of flag fancy=True - the rates are reported in percent with 2 decimals.

In [27]:
pr1.port_monthly_returns(fancy=True)

year,2015,2016,2017,2018,2019,2020,2021
month,,,,,,,
1,nan%,-1.66%,2.27%,3.37%,6.27%,1.78%,-2.03%
2,nan%,1.41%,3.44%,-0.23%,4.31%,-3.54%,-3.92%
3,nan%,1.59%,0.81%,0.27%,-0.71%,-6.60%,-2.25%
4,nan%,-0.86%,1.87%,0.89%,3.02%,8.80%,2.26%
5,nan%,2.36%,3.22%,4.71%,-2.92%,3.37%,2.91%
6,-1.92%,2.60%,-1.79%,-3.49%,7.39%,-0.50%,-2.39%
7,2.30%,3.43%,3.06%,0.42%,4.82%,7.57%,1.60%
8,-7.20%,-0.62%,2.61%,6.20%,-3.87%,-0.07%,nan%
9,-2.93%,0.10%,1.71%,-0.89%,-1.82%,-3.56%,nan%


### Portfolio returns per rolling period 

Additional reference info is included in the report. 

- `Droll` is the rolling date. Identifies the start of the rolling period, (Droll, next Droll].
- `Dfix` is the fixing date.
- `RR` simple rate of return per rolling period.
- portfolio weights for each rolling period.

> Note the use of flag fancy=True - the rates are reported in percent with 2 decimals.

In [28]:
pr1.port_period_returns(fancy=True)

,Droll,Dfix,RR,GLD,TLT,XLV,VGT,PSJ
0,2015-06-25,2015-06-24,-10.38,0.00,10.74,89.26,0.00,0.00
1,2015-09-25,2015-09-24,10.34,0.00,13.12,86.88,0.00,0.00
2,2015-12-28,2015-12-24,-1.18,0.00,34.84,18.07,11.61,35.48
3,2016-03-28,2016-03-24,2.98,0.00,40.35,38.22,21.04,0.39
4,2016-06-27,2016-06-24,7.08,0.00,42.00,29.92,12.00,16.09
5,2016-09-27,2016-09-26,-8.61,0.00,48.18,24.56,4.66,22.59
6,2016-12-27,2016-12-23,6.71,0.00,38.86,0.00,61.14,0.00
7,2017-03-28,2017-03-27,5.20,0.00,36.89,0.00,63.11,0.00
8,2017-06-27,2017-06-26,5.17,0.00,36.02,0.00,63.98,0.00
9,2017-09-26,2017-09-25,6.43,0.00,35.82,0.00,58.11,6.08


### Number of shares invested at each rebalancing period

In [29]:
pr1.get_nshares()

,GLD,PSJ,TLT,VGT,XLV
Droll,,,,,
2015-06-25,0,0,92,0,1179
2015-09-25,0,0,99,0,1166
2015-12-28,0,823,284,105,248
2016-03-28,0,9,306,193,557
2016-06-27,0,381,313,116,437
2016-09-27,0,504,376,42,365
2016-12-27,0,0,325,490,0
2017-03-28,0,0,320,495,0
2017-06-27,0,0,316,499,0


### Additional accounting information

Additional accounting information per rebalancing period.

- `Droll` rolling date. It identifies the rebalancing period.
- Invested number of shares per portfolio component
- `cash_invst` is the dollar value of the invested shares at the beginning of rebalancing period (using closing prices in the `Droll` date).
- `cash_roll` amount of cash rolled to next period. The source of this cash is twofold. First due to rounding to an integer number of shares and second due to price slippage between the weights computation time, `Dfix` date, and the execution (buy/sale) of the portfolio positions, `Droll` date. It can be positive or negative. In both cases it is a small value relative to the invested capital. It is the investor obligation to make the shortfall until next rolling date. 
- `cash_divd` is the amount of cash collected during the rebalancing period. It will be part of the investor capital in the next period.

In [30]:
pr1.get_account(fancy=True)

,GLD,PSJ,TLT,VGT,XLV,cash_invst,cash_roll,cash_divd
Droll,,,,,,,,
2015-06-25,0,0,92,0,1179,100443.58,-443.58,0.00
2015-09-25,0,0,99,0,1166,90014.54,2269.01,379.89
2015-12-28,0,823,284,105,248,99318.01,-55.76,455.37
2016-03-28,0,9,306,193,557,98147.26,112.80,240.43
2016-06-27,0,381,313,116,437,101067.91,248.58,470.56
2016-09-27,0,504,376,42,365,108222.71,-767.40,408.68
2016-12-27,0,0,325,490,0,98899.85,-257.99,531.23
2017-03-28,0,0,320,495,0,105532.30,-185.31,338.13
2017-06-27,0,0,316,499,0,111020.11,1642.15,247.17
